In [1]:
import os
import pandas as pd
import csv
import shutil
import numpy as np

# Section 1

Se crean las carpetas en blanco para guardar los nuevos archivos modificados

In [ ]:
#Crear carpetas en blanco

dir = "datos_Modificados"
if not os.path.exists(dir):
    os.makedirs(dir)

pollutants = ["CO", "NO", "NO2", "NOX", "O3", "PM2.5", "PM10", "RH", "SO2", "TMP", "WDR", "WSP"]
for pollutant in pollutants:
    pollutant_path = os.path.join(dir, pollutant)
    print(pollutant_path)
    os.makedirs(pollutant_path)
    year = 2005
    for y in range(20):
        year_path = os.path.join(pollutant_path, str(year+y))
        os.makedirs(year_path)
        print(year, year_path, y)



# Section 2

Los archivos descargados de la página aire y salud tiene una fila al inicio y dos al final que no pertenecen a la tabla de valores (el título e información pareciera de html),
por lo que se eliminan para que se puedan leer como dataframes con pandas, de otra forma causa error

In [4]:
def delete_rows(dir, new_dir, file_path):
    filas = []
    dataframe = pd.read_csv(file_path)
    num_rows = len(dataframe)
    print(num_rows, file_path)
    with open(file_path, 'r', newline='', encoding='utf-8') as archivo_csv:
        lector_csv = csv.reader(archivo_csv)
        for row in lector_csv:
            if "Promedio" in row[0]:
                print(row[0])    
            elif "<br" in row[0]:
                print(row[0])  
            elif "<b" in row[0]:
                print(row[0])          
            else:
                filas.append(row)


    subpath = file_path[len(dir):len(file_path)]
    new_path = os.path.join(new_dir, subpath)
    
    with open(new_path, 'w', newline='', encoding='utf-8') as archivo_csv_nuevo:
        escritor_csv = csv.writer(archivo_csv_nuevo)
        escritor_csv.writerows(filas)

    dataframe = pd.read_csv(new_path)
    num_rows = len(dataframe)
    print(num_rows)
    return dataframe, new_path


Se cambia el nombre de todos los archivos descargados (todos dicen Promedio_contaminanteX), por el nombre del contaminante_el mes_año

In [5]:
#dir = "datos_Modificados/"
dir = "Datos Aire y Salud/"
new_dir = "datos_Modificados/"


for actual_folder, folders, files in os.walk(dir):
    for file_name in files:
        file_path = os.path.join(actual_folder, file_name)
        if file_path.endswith(".csv"):
            sub_folders = file_path.split("/")
            sub_folders = sub_folders[len(sub_folders)-1].split("\\")
            variable = sub_folders[len(sub_folders)-3]
            year = sub_folders[len(sub_folders)-2]
            print(file_path, variable, year)

            try:
                dataframe, new_path = delete_rows(dir, new_dir, file_path)
            except:                
                subpath = file_path[len(dir):len(file_path)]
                path = os.path.join(new_dir, subpath)
                new_path = os.path.split(path)
                new_path = new_path[0]
                new_path = shutil.copy(file_path, new_path)
                dataframe = pd.read_csv(new_path)
            if file_name.startswith("Promedio"):
                months = {"01":"JAN", "02":"FEB", "03":"MAR", "04":"APR", "05":"MAY", "06":"JUN", "07":"JUL", "08":"AUG", "09":"SEP", "10":"OCT", "11":"NOV", "12":"DEC"}
                date = dataframe.iloc[0].Fecha
                num_month = date.split("-")
                month = months[num_month[1]]

                new_name = os.path.join(new_dir,variable)
                new_name = os.path.join(new_name, year)
                name = variable + "_" +month+"_"+year + ".csv"
                new_name = os.path.join(new_name, name)
                os.rename(new_path,new_name)


Datos Aire y Salud/CO\2005\Promedio horarios de co(1).csv CO 2005
647 Datos Aire y Salud/CO\2005\Promedio horarios de co(1).csv
Promedio horarios de coPromedio horarios de co
<br />
<b>Warning</b>:  session_destroy() expects exactly 0 parameters
644
Datos Aire y Salud/CO\2005\Promedio horarios de co(10).csv CO 2005
693 Datos Aire y Salud/CO\2005\Promedio horarios de co(10).csv
Promedio horarios de coPromedio horarios de co
<br />
<b>Warning</b>:  session_destroy() expects exactly 0 parameters
690
Datos Aire y Salud/CO\2005\Promedio horarios de co(11).csv CO 2005
716 Datos Aire y Salud/CO\2005\Promedio horarios de co(11).csv
Promedio horarios de coPromedio horarios de co
<br />
<b>Warning</b>:  session_destroy() expects exactly 0 parameters
713
Datos Aire y Salud/CO\2005\Promedio horarios de co(2).csv CO 2005
716 Datos Aire y Salud/CO\2005\Promedio horarios de co(2).csv
Promedio horarios de coPromedio horarios de co
<br />
<b>Warning</b>:  session_destroy() expects exactly 0 parameters


Se juntan POR CONTAMINANTE en un solo dataframe los datos de todos los meses y todos los año

In [6]:
new_dir = "datos_Modificados"
files_path = "Archivos_contaminantes_juntas"

if not os.path.exists(files_path):
    os.makedirs(files_path)
    
CO_dataframes = []
SO2_dataframes = []
NO_dataframes = []
NOX_dataframes = []
NO2_dataframes = []
PM10_dataframes = []
PM25_dataframes = []
O3_dataframes = []
RH_dataframes = []
WDR_dataframes = []
WSP_dataframes = []
TMP_dataframes = []


for actual_folder, folders, files in os.walk(new_dir):
    for file_name in files:
        file_path = os.path.join(actual_folder, file_name)
        df = pd.read_csv(file_path)
        if "CO" in file_name:
            CO_dataframes.append(df)
        elif "NOX" in file_name:
            NOX_dataframes.append(df)
        elif "NO2" in file_name:
            NO2_dataframes.append(df)
        elif "NO" in file_name:
            NO_dataframes.append(df)
        elif "O3" in file_name:
            O3_dataframes.append(df)
        elif "PM10" in file_name:
            PM10_dataframes.append(df)
        elif "PM2.5" in file_name:
            PM25_dataframes.append(df)
        elif "TMP" in file_name:
            TMP_dataframes.append(df)
        elif "WSP" in file_name:
            WSP_dataframes.append(df)
        elif "WDR" in file_name:
            WDR_dataframes.append(df)
        elif "RH" in file_name:
            RH_dataframes.append(df)
        elif "SO2" in file_name:
            SO2_dataframes.append(df)

df_final_CO = pd.concat(CO_dataframes, ignore_index=True, sort=False)
df_final_NO = pd.concat(NO_dataframes, ignore_index=True, sort=False)
df_final_NO2 = pd.concat(NO2_dataframes, ignore_index=True, sort=False)
df_final_NOX = pd.concat(NOX_dataframes, ignore_index=True, sort=False)
df_final_SO2 = pd.concat(SO2_dataframes, ignore_index=True, sort=False)
df_final_O3 = pd.concat(O3_dataframes, ignore_index=True, sort=False)
df_final_PM10 = pd.concat(PM10_dataframes, ignore_index=True, sort=False)
df_final_PM25 = pd.concat(PM25_dataframes, ignore_index=True, sort=False)
df_final_TMP = pd.concat(TMP_dataframes, ignore_index=True, sort=False)
df_final_WDR = pd.concat(WDR_dataframes, ignore_index=True, sort=False)
df_final_WSP = pd.concat(WSP_dataframes, ignore_index=True, sort=False)
df_final_RH = pd.concat(RH_dataframes, ignore_index=True, sort=False)


file_name_CO = os.path.join(files_path,"CO.csv")
file_name_O3 = os.path.join(files_path,"O3.csv")
file_name_NO = os.path.join(files_path,"NO.csv")
file_name_NO2 = os.path.join(files_path,"NO2.csv")
file_name_NOX = os.path.join(files_path,"NOX.csv")
file_name_PM10 = os.path.join(files_path,"PM10.csv")
file_name_PM25 = os.path.join(files_path,"PM25.csv")
file_name_TMP = os.path.join(files_path,"TMP.csv")
file_name_WDR = os.path.join(files_path,"WDR.csv")
file_name_WSP = os.path.join(files_path,"WSP.csv")
file_name_RH = os.path.join(files_path,"RH.csv")
file_name_SO2 = os.path.join(files_path,"SO2.csv")

df_final_CO.to_csv(file_name_CO, index=False)
df_final_SO2.to_csv(file_name_SO2, index=False)
df_final_NO.to_csv(file_name_NO, index=False)
df_final_NO2.to_csv(file_name_NO2, index=False)
df_final_NOX.to_csv(file_name_NOX, index=False)
df_final_O3.to_csv(file_name_O3, index=False)
df_final_PM10.to_csv(file_name_PM10, index=False)
df_final_PM25.to_csv(file_name_PM25, index=False)
df_final_TMP.to_csv(file_name_TMP, index=False)
df_final_WSP.to_csv(file_name_WSP, index=False)
df_final_WDR.to_csv(file_name_WDR, index=False)
df_final_RH.to_csv(file_name_RH, index=False)


Se cambian los datos vacios con valor de "nr" por NaN, y se separa la fecha en columnas de día, mes, año, y se crea una nueva con los datos de la fecha y la hora para poder ordenarlos de forma ascendente

# Section 3

Se guarda toda la info de todos los contaminantes en un solo dataframe

In [7]:
dir = "Archivos_contaminantes_juntas"
dir2 = "Archivos_contaminantes_juntas2"
final_filename = "data_15May_2024_all_stations.csv"
if not os.path.exists(dir2):
    os.makedirs(dir2)

files = os.listdir(dir)
dataframes = []
for file_name in files:
    file_dir = os.path.join(dir, file_name)
    df = pd.read_csv(file_dir, low_memory=False)
    df.replace('nr', np.nan, inplace=True)
    columns = df.columns
    for column in columns:
        if 'Unnamed' in column:
            df = df.drop(column, axis=1)
    df['Fecha'] = df['Fecha'].str.replace('/', '-')
    df[['dia', 'mes', 'year']] = df['Fecha'].str.split('-', expand=True)     #separamos la fecha para poder unirla con la hora
    df['Hora'] = df['Hora'].replace(24, 0)  #cambiamos las 24 horas por las 0 horas
    # Convertir la columna 'horas' al formato de tiempo de Pandas
    df['Hora'] = pd.to_datetime(df['Hora'], unit='h')
    # Formatear la columna 'horas' en el formato deseado
    df['Hora'] = df['Hora'].dt.strftime('%H:%M')
    df["fecha_hora"] = df['year'] + "/"+df['mes'] +"/"+df["dia"]+" "+df["Hora"]
    df= df.sort_values(by='fecha_hora') 
    
    df["pollutant"] = file_name.split(".")[0]
    columna_especifica="fecha_hora"
    # Verificar si la columna tiene valores NaN
    tiene_nan = df[columna_especifica].isnull().any()

    # Mostrar el resultado
    if tiene_nan:
        print(f"La columna '{columna_especifica}' tiene valores NaN.")
    else:
        print(f"La columna '{columna_especifica}' no tiene valores NaN.")

    file_dir2 = os.path.join(dir2, file_name)
    df.to_csv(file_dir2, index=False) 
    dataframes.append(df)
df_final = pd.concat(dataframes, ignore_index=False, sort=False)
df_final.to_csv(final_filename, index=False)
df = pd.read_csv(final_filename)
del df["LVI"]
del df["SS1"]
df.to_csv(final_filename , index=False)
df

La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.


,Fecha,Hora,ACO,AJU,ARA,ATI,AZC,CAM,CCA,CES,...,GAM,INN,MGH,MPA,SAC,dia,mes,year,fecha_hora,pollutant
0,01-01-2005,00:00,NaN,NaN,4.2,0.7,1.2,NaN,NaN,3.4,...,NaN,NaN,NaN,NaN,NaN,1,1,2005,2005/01/01 00:00,CO
1,01-01-2005,01:00,NaN,NaN,6.1,2.3,2.7,NaN,NaN,2.7,...,NaN,NaN,NaN,NaN,NaN,1,1,2005,2005/01/01 01:00,CO
2,01-01-2005,02:00,NaN,NaN,8.3,2.5,2.5,NaN,NaN,2.6,...,NaN,NaN,NaN,NaN,NaN,1,1,2005,2005/01/01 02:00,CO
3,01-01-2005,03:00,NaN,NaN,11.1,3.0,3.3,NaN,NaN,2.6,...,NaN,NaN,NaN,NaN,NaN,1,1,2005,2005/01/01 03:00,CO
4,01-01-2005,04:00,NaN,NaN,10.9,2.9,4.4,NaN,NaN,3.8,...,NaN,NaN,NaN,NaN,NaN,1,1,2005,2005/01/01 04:00,CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015452,15-05-2024,13:00,NaN,4.3,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,3.1,2.5,NaN,2.5,15,5,2024,2024/05/15 13:00,WSP
2015453,15-05-2024,14:00,NaN,4.2,NaN,NaN,NaN,NaN,NaN,NaN,...,2.1,3.8,2.8,NaN,3.9,15,5,2024,2024/05/15 14:00,WSP
2015454,15-05-2024,15:00,NaN,4.2,NaN,NaN,NaN,NaN,NaN,NaN,...,2.2,4.0,3.3,NaN,4.2,15,5,2024,2024/05/15 15:00,WSP
2015455,15-05-2024,16:00,NaN,3.9,NaN,NaN,NaN,NaN,NaN,NaN,...,4.5,5.5,4.7,NaN,4.8,15,5,2024,2024/05/15 16:00,WSP


In [8]:
df.keys()

Index(['Fecha', 'Hora', 'ACO', 'AJU', 'ARA', 'ATI', 'AZC', 'CAM', 'CCA', 'CES',
       'MON', 'CHO', 'COY', 'CUA', 'CUT', 'FAC', 'HGM', 'IMP', 'IZT', 'LAG',
       'LLA', 'LPR', 'MER', 'NEZ', 'PED', 'PLA', 'SAG', 'SFE', 'SJA', 'SUR',
       'TAC', 'TAH', 'TAX', 'TEC', 'TLA', 'TLI', 'TPN', 'UAX', 'UIZ', 'VAL',
       'VIF', 'XAL', 'AJM', 'BJU', 'FAR', 'GAM', 'INN', 'MGH', 'MPA', 'SAC',
       'dia', 'mes', 'year', 'fecha_hora', 'pollutant'],
      dtype='object')

Número de estaciones de las que se tiene información

In [9]:
len(df.keys())-7

48

# Section 4

Como solo recuperamos los datos del tráfico de algunas estaciones, guardamos únicamente los datos de esas estaciones

In [10]:
"""
estaciones_dir = "estacionesTRAFICO.CSV"
estaciones = pd.read_csv(estaciones_dir, encoding="Latin1")
estaciones.keys()
estaciones_contingencia = estaciones[estaciones.xTile_in>0]
print(len(estaciones_contingencia))
estaciones_contingencia.reset_index(drop=True)
"""

'\nestaciones_dir = "estacionesTRAFICO.CSV"\nestaciones = pd.read_csv(estaciones_dir, encoding="Latin1")\nestaciones.keys()\nestaciones_contingencia = estaciones[estaciones.xTile_in>0]\nprint(len(estaciones_contingencia))\nestaciones_contingencia.reset_index(drop=True)\n'

In [ ]:
"""
#estaciones_dir = "EstacionesContingencia.csv"
estaciones = pd.read_csv(estaciones_dir, encoding="Latin1")
if 
estaciones_contingencia = estaciones[estaciones.Monitorea_Contingencia==1.0]
estaciones_contingencia = estaciones_contingencia[estaciones_contingencia.Ya_activó_contingencias=="Ya_activó"]
estaciones_contingencia.head()
"""

In [7]:
#estaciones_contingencia = ["BJU", "FAC", "NEZ", "PED", "SAC"]

In [15]:
estaciones = df.keys()
borrar = ['Fecha', 'Hora', 'dia', 'mes', 'year', 'fecha_hora', 'pollutant']
estaciones = [elemento for elemento in estaciones if elemento not in borrar]
len(estaciones)

48

In [16]:
dir_to_save = "datos_por_estacion"
#dir_to_save = "all_data_2005xstation_31_03_2024"
if not os.path.exists(dir_to_save):
    os.makedirs(dir_to_save)
    
#final_filename = "all_data_2005-2024_all_stations_prev_29_02_2024.csv"
df = pd.read_csv(final_filename)
pollutants = ["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP"]

for estacion in estaciones:
  columnas_interes = ["fecha_hora", "pollutant"]
  columnas_interes.append(estacion)
  datos_modelos = df[columnas_interes]
  datos_modelos = datos_modelos.rename(columns={estacion: "value"})
  dataframes = []
  for pollutant in pollutants:
    data = datos_modelos[datos_modelos.pollutant==pollutant]
    data = data.set_index("fecha_hora")
    data.index.name = 'date'
    data.drop('pollutant', axis=1, inplace=True)
    data = data.rename(columns={"value":pollutant})
    dataframes.append(data)

  df_merged = pd.DataFrame()
  columnas = ['date']
  df_merged = pd.DataFrame(columns=columnas)
  for dataframe in dataframes:
    df_merged = pd.merge(df_merged, dataframe, on='date', how='outer')

  df_merged = df_merged.reset_index()
  df_merged = df_merged.drop(columns=["index"])
  df_merged_sin_fecha = df_merged.drop(columns=["date"])
  indices_filas_vacias = df_merged_sin_fecha[df_merged_sin_fecha.isna().all(axis=1)].index
  df_filtrado = df_merged.drop(indices_filas_vacias)
  df_filtrado = df_filtrado.set_index("date")
  df_filtrado= df_filtrado.sort_values(by='date') 
  df_filtrado = df_filtrado.reset_index()
  name = estacion + ".csv"
  dir_file = os.path.join(dir_to_save,name)
  df_filtrado.to_csv(dir_file, index=False)

In [17]:
df_filtrado

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
0,2019/03/01 00:00,0.30,NaN,NaN,NaN,NaN,NaN,29.0,NaN,NaN,NaN,171.0,1.1
1,2019/03/01 01:00,0.20,1.0,17.0,15.0,45.0,NaN,21.0,NaN,1.0,NaN,150.0,2.5
2,2019/03/01 02:00,0.40,3.0,34.0,30.0,23.0,NaN,22.0,NaN,1.0,NaN,156.0,0.8
3,2019/03/01 03:00,0.60,7.0,46.0,38.0,12.0,NaN,29.0,NaN,1.0,NaN,200.0,1.0
4,2019/03/01 04:00,0.70,15.0,63.0,48.0,3.0,NaN,32.0,NaN,1.0,NaN,292.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57079,2024/05/15 13:00,0.59,2.0,11.0,9.0,85.0,NaN,48.0,18.0,1.0,30.0,10.0,2.5
57080,2024/05/15 14:00,0.56,1.0,8.0,6.0,81.0,NaN,50.0,19.0,1.0,30.0,121.0,3.9
57081,2024/05/15 15:00,0.53,2.0,7.0,5.0,73.0,NaN,61.0,18.0,1.0,30.0,116.0,4.2
57082,2024/05/15 16:00,0.79,2.0,13.0,12.0,76.0,NaN,NaN,26.0,1.0,28.0,211.0,4.8


Se borran las carpetan que ya no nos sirven

In [18]:
dir = "Archivos_contaminantes_juntas"
shutil.rmtree(dir)
dir = "Archivos_contaminantes_juntas2"
shutil.rmtree(dir)
#dir = "datos_por_estacion"
#shutil.rmtree(dir)

#os.remove(final_filename)